In [2]:
!pip install pyTelegramBotAPI SpeechRecognition pydub config

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.8/232.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 31.3 MB/s eta 0:00:00
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.13.0-py3-none-any.whl size=214529 sha256=2d1a5a8c86cf76b7ecc1c8481d2e7965cd07d13966041240d1ac470aef61ef58
  Stored in directory: /root/.cache/pip/wheels/f7/1a/3b/41e1ce64625e3f11567085dc79043d9fa573a2cc39f9938556
Successfully built pyTelegramBotAPI


In [8]:
import os
import telebot
from telebot import types
import speech_recognition
from pydub import AudioSegment
from PIL import Image, ImageEnhance, ImageFilter
import config


token = '6609113829:AAGvThcOtkftKx5fU7sHoVFi9jMYfG6HRf4'

bot = telebot.TeleBot(token)

BLUR_MODE = 2
EMBOSS_MODE = 1

filter_mode = 1

def transform_image_EMBOSS(filename):
    # Функция обработки изображения
    source_image = Image.open(filename)
    enhanced_image = source_image.filter(ImageFilter.EMBOSS)
    enhanced_image = enhanced_image.convert('RGB')
    enhanced_image.save(filename)
    return filename


def transform_image_BLUR(filename):
    # Функция обработки изображения
    source_image = Image.open(filename)
    enhanced_image = source_image.filter(ImageFilter.BLUR)
    enhanced_image = enhanced_image.convert('RGB')
    enhanced_image.save(filename)
    return filename



@bot.message_handler(commands=['start'])
def say_hi(message):
    # Функция, отправляющая "Привет" в ответ на команду /start
    bot.send_message(message.chat.id, 'Привет!'.format(message.from_user))


@bot.message_handler(commands=['emboss'])
def make_emboss(message):
  global filter_mode
  filter_mode = EMBOSS_MODE


@bot.message_handler(commands=['blur'])
def make_blur(message):
  global filter_mode
  filter_mode = BLUR_MODE


@bot.message_handler(content_type=['text'])
def send_photo(message):
    global filter_mode
    if(message.text == "EMBOSS"):
        filter_mode = EMBOSS_MODE
    elif(message.text == "BLUR"):
        filter_mode = BLUR_MODE


@bot.message_handler(content_types=['photo'])
def choose_filter(message):
    global filter_mode

    # Функция отправки обработанного изображения
    if filter_mode == EMBOSS_MODE:
        EMBOSS(message)
    elif filter_mode == BLUR_MODE:
        BLUR(message)


def BLUR(message):
    # Функция отправки обработанного изображения
    file_id = message.photo[-1].file_id
    filename = download_file(bot, file_id)

    # Трансформируем изображение
    transform_image_BLUR(filename)

    image = open(filename, 'rb')
    bot.send_photo(message.chat.id, image)
    image.close()

    # Не забываем удалять ненужные изображения
    if os.path.exists(filename):
        os.remove(filename)


def EMBOSS(message):
    # Функция отправки обработанного изображения
    file_id = message.photo[-1].file_id
    filename = download_file(bot, file_id)

    # Трансформируем изображение
    transform_image_EMBOSS(filename)

    image = open(filename, 'rb')
    bot.send_photo(message.chat.id, image)
    image.close()

    # Не забываем удалять ненужные изображения
    if os.path.exists(filename):
        os.remove(filename)


def oga2wav(filename):
    # Конвертация формата файлов
    new_filename = filename.replace('.oga', '.wav')
    audio = AudioSegment.from_file(filename)
    audio.export(new_filename, format='wav')
    return new_filename


def recognize_speech(oga_filename):
    # Перевод голоса в текст + удаление использованных файлов
    wav_filename = oga2wav(oga_filename)
    recognizer = speech_recognition.Recognizer()

    with speech_recognition.WavFile(wav_filename) as source:
        wav_audio = recognizer.record(source)

    text = recognizer.recognize_google(wav_audio, language='ru')

    if os.path.exists(oga_filename):
        os.remove(oga_filename)

    if os.path.exists(wav_filename):
        os.remove(wav_filename)

    return text


def download_file(bot, file_id):
    # Скачивание файла, который прислал пользователь
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    filename = file_id + file_info.file_path
    filename = filename.replace('/', '_')
    with open(filename, 'wb') as f:
        f.write(downloaded_file)
    return filename


@bot.message_handler(content_types=['voice'])
def transcript(message):
    # Функция, отправляющая текст в ответ на голосовое
    filename = download_file(bot, message.voice.file_id)
    text = recognize_speech(filename)
    bot.send_message(message.chat.id, text)


# Запускаем бота. Он будет работать до тех пор, пока работает ячейка (крутится значок слева).
# Остановим ячейку - остановится бот
bot.polling()